# 🏠 Análisis de datos inmobiliarios

Por Coke y Alejandro

Este cuaderno analiza un conjunto de datos inmobiliarios, explora estadísticas univariables y bivariables, realiza pruebas de normalidad, estima parámetros de población y realiza estadísticas inferenciales, incluidas pruebas de hipótesis y ANOVA.

Dataset: [Datos inmobiliarios de Londres 2024](https://www.kaggle.com/datasets/kanchana1990/real-estate-data-london-2024).

Fuente: [Kaggle](https://www.kaggle.com/).

Origen de los datos: [Rightmove](https://www.rightmove.co.uk/), Noviembre de 2024.

### 🗂️ **Dependencias**

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import re

import matplotlib as plt
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
from collections import Counter

import scipy
import scipy.stats as st
from scipy import stats
import scipy.stats as stats
from scipy.stats import skew
import nbformat

## ⚙️ **Configuración Inicial**

* Carga previa de data.csv en la raíz de Archivos.
* Exploración inicial
* Limpieza del dataset
* Generación de subconjunto
* Inspección de outliers


### 📥 Importación de Datos

Importación con Pandas y asignación a la variable **data**.

In [3]:
data = pd.read_csv("data/data.csv")

### 🔍 Exploración Inicial

* Vista previa
* Informe Y Data
* Informe Sweetviz

Iofrmación y muestra de cabecera del dataset para obtener una idea aproximada de los datos disponibles.

In [4]:
data.describe()

,sizeSqFeetMax,bedrooms,bathrooms
count,869.000000,1003.000000,984.000000
mean,5232.871116,5.111665,4.648374
std,11796.770144,3.264992,3.085809
min,425.000000,1.000000,1.000000
25%,2885.000000,3.000000,3.000000
50%,3834.000000,5.000000,4.000000
75%,5745.000000,6.000000,5.000000
max,336989.000000,66.000000,66.000000


In [5]:
data.head()

,addedOn,title,descriptionHtml,propertyType,sizeSqFeetMax,bedrooms,bathrooms,listingUpdateReason,price
0,10/10/2024,"8 bedroom house for sale in Winnington Road, H...","This magnificent home, set behind security gat...",House,16749.0,8.0,8.0,new,"£24,950,000"
1,Reduced on 24/10/2024,"7 bedroom house for sale in Brick Street, Mayf...","In the heart of exclusive Mayfair, this majest...",House,12960.0,7.0,7.0,price_reduced,"£29,500,000"
2,Reduced on 22/02/2024,6 bedroom terraced house for sale in Chester S...,A freehold home that gives you everything you ...,Terraced,6952.0,6.0,6.0,price_reduced,"£25,000,000"
3,08/04/2024,6 bedroom detached house for sale in Winningto...,A magnificent bespoke residence set behind sec...,Detached,16749.0,6.0,6.0,new,"£24,950,000"
4,Reduced on 11/07/2023,8 bedroom detached house for sale in St. John'...,"With its village like ambiance, elegant regenc...",Detached,10241.0,8.0,10.0,price_reduced,"£24,950,000"


### ✨ Limpieza del dataset

* Entradas `NaN` y duplicadas descartadas (2.3%)
* Columnas innecesarias descartadas (3)
* Columna `title` reformateada a `ZIP`
* Formateo columna `price`
* Creación de columna `pricePerSqFoot`

#### ⛔️ Entradas NaN y duplicadas

Se descartan las filas que contengan algún valor NaN y las duplicadas ya que no representan un porcentaje significativo de la muestra.

* 20 entradas de 869
* 2.3%

In [6]:
data=data.dropna()
data=data.drop_duplicates().reset_index(drop=True)

In [7]:
data

,addedOn,title,descriptionHtml,propertyType,sizeSqFeetMax,bedrooms,bathrooms,listingUpdateReason,price
0,10/10/2024,"8 bedroom house for sale in Winnington Road, H...","This magnificent home, set behind security gat...",House,16749.0,8.0,8.0,new,"£24,950,000"
1,Reduced on 24/10/2024,"7 bedroom house for sale in Brick Street, Mayf...","In the heart of exclusive Mayfair, this majest...",House,12960.0,7.0,7.0,price_reduced,"£29,500,000"
2,Reduced on 22/02/2024,6 bedroom terraced house for sale in Chester S...,A freehold home that gives you everything you ...,Terraced,6952.0,6.0,6.0,price_reduced,"£25,000,000"
3,08/04/2024,6 bedroom detached house for sale in Winningto...,A magnificent bespoke residence set behind sec...,Detached,16749.0,6.0,6.0,new,"£24,950,000"
4,Reduced on 11/07/2023,8 bedroom detached house for sale in St. John'...,"With its village like ambiance, elegant regenc...",Detached,10241.0,8.0,10.0,price_reduced,"£24,950,000"
...,...,...,...,...,...,...,...,...,...
844,04/06/2024,7 bedroom semi-detached house for sale in Lyfo...,"This spectacular seven bedroom, semi-detached,...",Semi-Detached,7454.0,7.0,4.0,new,"£6,500,000"
845,04/06/2024,7 bedroom semi-detached house for sale in Lyfo...,An exquisite seven bedroom house offering luxu...,Semi-Detached,7454.0,7.0,5.0,new,"£6,500,000"
846,Added today,3 bedroom apartment for sale in Battersea Powe...,NO STAMP DUTY This stunning penthouse apartmen...,Apartment,2601.0,3.0,4.0,new,"£6,500,000"
847,28/08/2024,"3 bedroom apartment for sale in Vicarage Gate,...","Welcome to 2 Vicarage Gate House, a sophistica...",Apartment,2508.0,3.0,3.0,new,"£6,500,000"


#### ⛔️ Columnas innecesarias

Se eliminan las columnas innecesarias:


*   addedOn: No importa en qué fecha se añadió.
*   descriptionHtml: La descripción detallada no aporta información útil para el análisis.
*   listingUpdateReason: El motivo por el que está en la lista tampoco interesa en este caso.




In [8]:
data = data.drop(columns=["addedOn", "descriptionHtml", "listingUpdateReason"])

#### 💄 Columna ZIP

De la columna "title" se extrae "Location" y a su vez, de "Location" obtenemos el "ZIP".

A continuación se eliminan ambas columnas y conservamos únicamente el "ZIP"

In [9]:
data["Location"] = data["title"].str.split("in ", n=1).str[1]
data["ZIP"] = data["Location"].str.split(", ").str[-1]

In [10]:
data=data.drop(["Location","title"], axis=1)

#### 💄 Columna price

Formateo de la columna "price" para quitar símbolos y convertirla en numérica.

In [11]:
data["price"] = pd.to_numeric(data["price"].str.replace('£', '', regex=False).str.replace(',', '', regex=False), errors='coerce')

Además, reduciremos el precio en mil libras para facilitar la visualización

In [12]:
data["price(M)"] = data["price"] / 1000000
data=data.drop("price", axis=1)

#### ✅ Columna pricePerSqFoot

In [13]:
data["pricePerSqFoot(k)"] = round((data["price(M)"] * 1000) / data["sizeSqFeetMax"], 2)

#### 👀 Comprobación

Comprobación de la nueva columna "ZIP" y del formato de "price"

In [14]:
data.head()

,propertyType,sizeSqFeetMax,bedrooms,bathrooms,ZIP,price(M),pricePerSqFoot(k)
0,House,16749.0,8.0,8.0,N2,24.95,1.49
1,House,12960.0,7.0,7.0,W1J,29.50,2.28
2,Terraced,6952.0,6.0,6.0,SW1W,25.00,3.60
3,Detached,16749.0,6.0,6.0,N2,24.95,1.49
4,Detached,10241.0,8.0,10.0,NW8,24.95,2.44


In [15]:
data.describe()

,sizeSqFeetMax,bedrooms,bathrooms,price(M),pricePerSqFoot(k)
count,849.000000,849.000000,849.000000,849.000000,849.000000
mean,5122.775029,4.963486,4.584217,11.333114,2.765630
std,11850.272127,2.461464,2.277926,6.757395,1.453578
min,425.000000,1.000000,1.000000,0.475000,0.030000
25%,2885.000000,3.000000,3.000000,7.500000,1.940000
50%,3794.000000,5.000000,4.000000,9.250000,2.550000
75%,5624.000000,6.000000,5.000000,13.000000,3.360000
max,336989.000000,36.000000,34.000000,80.000000,19.780000


### 📚 Subconjunto

* Creación de columna `propertyGlobalType`
* Creación de tipos globales `House` y `Flat`
* Creación de subset `subset`


Con el objetivo de sacar conclusiones sobre los tipos de inmuebles generamos 2 tipos globales agrupando 3 tipos de propiedad cada uno:
* House
  * House
  * Detached
  * Terraced
* Flat
  * Flat
  * Apartment
  * Penthouse

In [16]:
data['propertyGlobalType']= np.where(data['propertyType'].isin(['Apartment', 'Penthouse', 'Flat']), 'Flat',
    np.where(data['propertyType'].isin(['House', 'Terraced', 'Detached']),'House',None))

In [17]:
subset = data[data['propertyGlobalType'].notna()]
subset = subset[["ZIP","propertyType","propertyGlobalType","bedrooms","bathrooms","sizeSqFeetMax","pricePerSqFoot(k)","price(M)"]]

In [18]:
subset.head()

,ZIP,propertyType,propertyGlobalType,bedrooms,bathrooms,sizeSqFeetMax,pricePerSqFoot(k),price(M)
0,N2,House,House,8.0,8.0,16749.0,1.49,24.95
1,W1J,House,House,7.0,7.0,12960.0,2.28,29.50
2,SW1W,Terraced,House,6.0,6.0,6952.0,3.60,25.00
3,N2,Detached,House,6.0,6.0,16749.0,1.49,24.95
4,NW8,Detached,House,8.0,10.0,10241.0,2.44,24.95


### 🏴‍☠️ Outliers

Por defecto procedemos a descartar outliers que podrían complicar las predicciones:

* Top 3 por tamaño, descartados.
* Más de 15 baños o habitaciones, descartados.

In [19]:
subset_by_sizeSqFeetMax = subset.sort_values(by='sizeSqFeetMax', ascending=False)
subset_by_sizeSqFeetMax.head(10)

,ZIP,propertyType,propertyGlobalType,bedrooms,bathrooms,sizeSqFeetMax,pricePerSqFoot(k),price(M)
426,SW8,Penthouse,Flat,5.0,4.0,336989.0,0.03,9.60
813,SW10,Apartment,Flat,4.0,4.0,36909.0,0.18,6.50
203,N2,Detached,House,8.0,8.0,31376.0,0.40,12.50
39,W1B,Terraced,House,10.0,10.0,20987.0,3.10,65.00
24,TW1,Detached,House,9.0,12.0,18837.0,1.59,29.95
30,N6,Detached,House,10.0,9.0,18238.0,1.75,32.00
0,N2,House,House,8.0,8.0,16749.0,1.49,24.95
3,N2,Detached,House,6.0,6.0,16749.0,1.49,24.95
77,N2,Detached,House,6.0,9.0,16748.0,1.49,24.95
34,SW1E,Terraced,House,9.0,9.0,15845.0,2.84,45.00


Descartamos las tres primeras propiedades con más tamaño.

In [20]:
subset = subset.drop(index=[426, 813, 203])

In [21]:
subset_by_sizeSqFeetMax = subset.sort_values(by='sizeSqFeetMax', ascending=True)
subset_by_sizeSqFeetMax.head(10)

,ZIP,propertyType,propertyGlobalType,bedrooms,bathrooms,sizeSqFeetMax,pricePerSqFoot(k),price(M)
808,N5,Apartment,Flat,1.0,1.0,425.0,1.29,0.550
428,SW8,Penthouse,Flat,5.0,4.0,593.0,16.19,9.600
602,NW6,Apartment,Flat,2.0,1.0,642.0,1.17,0.750
592,W2,Apartment,Flat,1.0,1.0,696.0,1.51,1.050
302,E14,Flat,Flat,2.0,2.0,734.0,0.85,0.625
575,SE8,Apartment,Flat,2.0,2.0,871.0,0.63,0.550
834,SW8,Penthouse,Flat,5.0,4.0,872.0,7.45,6.500
135,N10,Apartment,Flat,2.0,1.0,900.0,0.81,0.725
344,SW17,House,House,3.0,1.0,911.0,0.87,0.795
345,E14,Flat,Flat,2.0,2.0,915.0,0.57,0.525




---



In [22]:
subset_by_bedrooms = subset.sort_values(by=['bedrooms',"bathrooms"], ascending=False)
subset_by_bedrooms.head(10)

,ZIP,propertyType,propertyGlobalType,bedrooms,bathrooms,sizeSqFeetMax,pricePerSqFoot(k),price(M)
714,SE18,Apartment,Flat,30.0,30.0,13096.0,0.55,7.200
46,W8,House,House,29.0,19.0,11238.0,1.65,18.500
44,W8,Terraced,House,14.0,20.0,12575.0,1.47,18.500
590,NW2,House,House,13.0,11.0,10192.0,0.78,8.000
589,NW2,Detached,House,13.0,10.0,10127.0,0.79,8.000
381,SW7,House,House,12.0,8.0,8791.0,1.13,9.975
102,W14,Detached,House,12.0,6.0,6497.0,2.69,17.500
724,W1U,Terraced,House,12.0,3.0,8793.0,0.80,7.000
131,NW3,Detached,House,11.0,9.0,6071.0,2.79,16.950
651,SW19,Detached,House,11.0,6.0,5680.0,1.32,7.495


Descartamos aquellas propiedades con más de 15 baños o más de 15 habitaciones.

In [23]:
subset = subset.drop(index=[714, 46, 44])



---



In [24]:
subset_by_pricePerSqFoot = subset.sort_values(by='pricePerSqFoot(k)', ascending=False)
subset_by_pricePerSqFoot.head(5)

,ZIP,propertyType,propertyGlobalType,bedrooms,bathrooms,sizeSqFeetMax,pricePerSqFoot(k),price(M)
428,SW8,Penthouse,Flat,5.0,4.0,593.0,16.19,9.60
40,SW7,Penthouse,Flat,5.0,5.0,9437.0,8.48,80.00
455,W1K,Penthouse,Flat,5.0,2.0,1055.0,8.48,8.95
188,W1U,Apartment,Flat,3.0,3.0,1959.0,7.63,14.95
662,SW1X,Flat,Flat,1.0,2.0,990.0,7.58,7.50


In [25]:
subset_SW8= subset[subset['ZIP']=="SW8"]
subset_by_pricePerSqFoot = subset_SW8.sort_values(by='pricePerSqFoot(k)', ascending=False)
subset_by_pricePerSqFoot.head(3)

,ZIP,propertyType,propertyGlobalType,bedrooms,bathrooms,sizeSqFeetMax,pricePerSqFoot(k),price(M)
428,SW8,Penthouse,Flat,5.0,4.0,593.0,16.19,9.600
834,SW8,Penthouse,Flat,5.0,4.0,872.0,7.45,6.500
342,SW8,Apartment,Flat,5.0,6.0,3114.0,3.45,10.747


Descartamos una propiedad que duplica a la segunda con el precio por pie cuadrado más elevado.

In [26]:
subset = subset.drop(index=428)

In [27]:
subset_by_pricePerSqFoot = subset.sort_values(by='pricePerSqFoot(k)', ascending=False)
subset_by_pricePerSqFoot.tail(10)

,ZIP,propertyType,propertyGlobalType,bedrooms,bathrooms,sizeSqFeetMax,pricePerSqFoot(k),price(M)
679,N21,Detached,House,6.0,6.0,12500.0,0.60,7.500
614,N21,Detached,House,6.0,6.0,12500.0,0.60,7.500
477,E16,Flat,Flat,2.0,2.0,992.0,0.58,0.575
491,N2,Detached,House,8.0,7.0,14999.0,0.58,8.750
345,E14,Flat,Flat,2.0,2.0,915.0,0.57,0.525
534,N20,Flat,Flat,3.0,2.0,1231.0,0.56,0.695
698,N1,House,House,6.0,6.0,12809.0,0.54,6.950
785,N21,Detached,House,5.0,3.0,2565.0,0.47,1.200
499,NW4,Flat,Flat,2.0,1.0,1012.0,0.47,0.475
843,N2,Detached,House,6.0,10.0,14770.0,0.44,6.500


In [28]:
subset_by_price = subset.sort_values(by='price(M)', ascending=False)
subset_by_price

,ZIP,propertyType,propertyGlobalType,bedrooms,bathrooms,sizeSqFeetMax,pricePerSqFoot(k),price(M)
40,SW7,Penthouse,Flat,5.0,5.0,9437.0,8.48,80.000
39,W1B,Terraced,House,10.0,10.0,20987.0,3.10,65.000
37,SW1W,House,House,6.0,6.0,13067.0,3.60,47.000
34,SW1E,Terraced,House,9.0,9.0,15845.0,2.84,45.000
33,SW1X,Terraced,House,8.0,8.0,9049.0,4.20,38.000
...,...,...,...,...,...,...,...,...
477,E16,Flat,Flat,2.0,2.0,992.0,0.58,0.575
808,N5,Apartment,Flat,1.0,1.0,425.0,1.29,0.550
575,SE8,Apartment,Flat,2.0,2.0,871.0,0.63,0.550
345,E14,Flat,Flat,2.0,2.0,915.0,0.57,0.525


Descartamos el inmueble con el precio más elevado, que marca más del doble del siguiente de su tipo global.

In [29]:
subset = subset.drop(index=40)

## 💾 Descarga de subset

Innecesario si se ejecuta toda la sesión.

In [149]:
subset.to_csv('data/subset.csv', index=False)

## 📤 Recarga de subset

Innecesario si se ejecuta toda la sesión.

In [30]:
subset = pd.read_csv("data/subset.csv")

## 📊 **Descripción de Variables y analisis univariante**

Además de comprobar la normalidad de las variables en las que sea posible.

### Cuantitativas

No presentan distribución normal.

#### **sizeSqFeetMax**

* Descripción: Área del inmueble en pies cuadrados.
* Tipo de variable: Cuantitativa continua.
* Normalidad: No (F de Fisher-Snedecor?)

In [116]:
subset["sizeSqFeetMax"].describe()


count      735.000000
mean      4559.235374
std       2845.480646
min        425.000000
25%       2851.000000
50%       3686.000000
75%       5518.500000
max      20987.000000
Name: sizeSqFeetMax, dtype: float64

In [117]:
fig = px.histogram(
    subset, 
    x="sizeSqFeetMax", 
    nbins=100, 
    title="Distribución por área",
)

mean_value = subset["sizeSqFeetMax"].mean()
median_value = subset["sizeSqFeetMax"].median()

fig.add_vline(
    x=mean_value, 
    line=dict(color='black', dash='dash'), 
    annotation_text="Mean", 
    annotation_position="top right"
)
fig.add_vline(
    x=median_value, 
    line=dict(color='red', dash='dash'), 
    annotation_text="Median", 
    annotation_position="top left"
)

# Show the legend
fig.update_layout(legend_title_text="Metrics")
fig.show()


Dada la diferencia entre media y mediana de aprocximadamente 1300 pies y la forma de la distribución, se concluye que la distribución de sizeSqFeetMax no es normal.

Podría tratarse de una Distribución F de Fisher-Snedecor.

In [118]:
Size=subset["sizeSqFeetMax"]
print("Asimetria: "+ str(skew(Size, axis=0, bias=True)))
print("Curtosis: "+ str(stats.kurtosis(Size, axis=0, fisher=True, bias=True)))

Asimetria: 1.9733182365290558
Curtosis: 5.1703746104766015


La asimetría a la izquierda ya era bastante apreciable aunque la curtosis es más alta de lo esperado. A pesar de ello, su cercanía al 0 indica parecido con una distribución normal.

#### **price**

* Descripción: Precio del inmueble en libras.
* Tipo de variable: Cuantitativa continua.
* Normalidad: No (F de Fisher-Snedecor?)


In [119]:
subset["price(M)"].describe()


count    735.000000
mean      10.958573
std        5.942697
min        0.475000
25%        7.500000
50%        9.000000
75%       12.850000
max       65.000000
Name: price(M), dtype: float64

In [120]:
fig = px.histogram(
    subset, 
    x="price(M)", 
    nbins=100, 
    title="Distribución por precio",
)

mean_value = subset["price(M)"].mean()
median_value = subset["price(M)"].median()

fig.add_vline(
    x=mean_value, 
    line=dict(color='black', dash='dash'), 
    annotation_text="Mean", 
    annotation_position="top right"
)
fig.add_vline(
    x=median_value, 
    line=dict(color='red', dash='dash'), 
    annotation_text="Median", 
    annotation_position="top left"
)

# Show the legend
fig.update_layout(legend_title_text="Metrics")
fig.show()


A pesar de tener un suelo en 6.5M, la distribucion es muy similar a `sizeSqFeetMax`, se concluye que la distribución de sizeSqFeetMax no es normal.


In [121]:
price=subset["price(M)"]
print("Asimetria: "+ str(skew(price, axis=0, bias=True)))
print("Curtosis: "+ str(stats.kurtosis(price, axis=0, fisher=True, bias=True)))

Asimetria: 2.66270019990273
Curtosis: 13.484118324222077


La asimetría y no normalidad ya eran extremadamente obras antes del test por lo que no indagaremos en estos resultados, más allá de destacar lo alto que es el valor de curtosis.

#### **pricePerSqFoot**

* Descripción: Precio del pie cuadrado.
* Tipo de variable: Cuantitativa continua.
* Normalidad: No (F de Fisher-Snedecor?)



In [122]:
subset["pricePerSqFoot(k)"].describe()


count    735.000000
mean       2.757374
std        1.257427
min        0.440000
25%        1.945000
50%        2.570000
75%        3.385000
max        8.480000
Name: pricePerSqFoot(k), dtype: float64

In [123]:
fig = px.histogram(
    subset, 
    x="pricePerSqFoot(k)", 
    nbins=40, 
    title="Distribución por precio",
)

mean_value = subset["pricePerSqFoot(k)"].mean()
median_value = subset["pricePerSqFoot(k)"].median()

fig.add_vline(
    x=mean_value, 
    line=dict(color='black', dash='dash'), 
    annotation_text="Mean", 
    annotation_position="top right"
)
fig.add_vline(
    x=median_value, 
    line=dict(color='red', dash='dash'), 
    annotation_text="Median", 
    annotation_position="top left"
)

# Show the legend
fig.update_layout(legend_title_text="Metrics")
fig.show()


En este caso la media y la mediana están relativamente cerca una de otra y la cola es lo bastante pequeña como para causar dudas sobre su normalidad.

In [124]:
PpF=subset["pricePerSqFoot(k)"]
print("Asimetria: "+ str(skew(PpF, axis=0, bias=True)))
print("Curtosis: "+ str(stats.kurtosis(PpF, axis=0, fisher=True, bias=True)))

Asimetria: 1.0625133634495507
Curtosis: 1.8818854050464422


La asimetría es más alta de o esperado, pero comprensible puesto que esta variable está directamente relacionada con `price`. Por otro lado, la curtosis es la más pequeña hasta ahora.

#### **bedrooms**

* Descripción: Número de habitaciones.
* Tipo de variable: Cuantitativa discreta.
* Normalidad: No


In [125]:
subset["bedrooms"].describe()


count    735.000000
mean       4.779592
std        1.866926
min        1.000000
25%        3.000000
50%        5.000000
75%        6.000000
max       13.000000
Name: bedrooms, dtype: float64

In [126]:
bedroom_counts = subset["bedrooms"].value_counts().reset_index()
bedroom_counts.columns = ["bedrooms", "count"]

fig = px.bar(
    bedroom_counts,
    x="bedrooms",
    y="count",
    title="Distribución por número de habitaciones",
    labels={"bedrooms": "Number of Bedrooms", "count": "Count"}
)

mean_value = subset["bedrooms"].mean()
median_value = subset["bedrooms"].median()

fig.add_vline(
    x=mean_value, 
    line=dict(color='black', dash='dash'), 
    annotation_text="Mean", 
    annotation_position="top right"
)
fig.add_vline(
    x=median_value, 
    line=dict(color='red', dash='dash'), 
    annotation_text="Median", 
    annotation_position="top left"
)

fig.update_layout(
    xaxis_title="Number of Bedrooms",
    yaxis_title="Count",
    legend_title_text="Metrics"
)

fig.show()


#### **bathrooms**

* Descripción: Número de baños.
* Tipo de variable: Cuantitativa discreta.
* Normalidad: No

In [127]:
subset["bathrooms"].describe()


count    735.000000
mean       4.421769
std        1.722290
min        1.000000
25%        3.000000
50%        4.000000
75%        5.000000
max       12.000000
Name: bathrooms, dtype: float64

In [128]:
bathrooms_counts = subset["bathrooms"].value_counts().reset_index()
bathrooms_counts.columns = ["bathrooms", "count"]

fig = px.bar(
    bathrooms_counts,
    x="bathrooms",
    y="count",
    title="Distribución por número de baños",
    labels={"bathrooms": "Number of Bathrooms", "count": "Count"}
)

mean_value = subset["bathrooms"].mean()
median_value = subset["bathrooms"].median()

fig.add_vline(
    x=mean_value, 
    line=dict(color='black', dash='dash'), 
    annotation_text="Mean", 
    annotation_position="top right"
)
fig.add_vline(
    x=median_value, 
    line=dict(color='red', dash='dash'), 
    annotation_text="Median", 
    annotation_position="top left"
)

fig.update_layout(
    xaxis_title="Number of Bathrooms",
    yaxis_title="Count",
    legend_title_text="Metrics"
)

fig.show()


### Categóricas

#### **ZIP**

*   Descripción: Código postal.
*   Tipo de variable: Categórica nominativa.
*   Distribución: Concentración en centro-oeste de Londres

In [129]:
subset["ZIP"].unique()

array(['N2', 'W1J', 'SW1W', 'NW8', 'SW1X', 'NW1', 'TW1', 'SW10', 'N6',
       'W11', 'SW1E', 'W1B', 'SW1A', 'W1K', 'SW7', 'NW3', 'W1H', 'SW3',
       'W8', 'W14', 'Pembroke Gardens,London,W8', 'W9', 'EC1V', 'E20',
       'W2', 'NW7', 'N10', 'W1S', 'WC2N', 'SW19', "St John's Wood NW8",
       'Hampstead NW3', 'W1U', 'SW16', 'South Kensington SW7', 'W1G',
       'SW6', 'SE1', 'WC2R', 'NW11', 'SE13', 'W1W', 'SW15', 'E14', 'SW8',
       'SW1P', 'Nine Elms SW8', 'Chelsea SW3', 'Kensington W8', 'SW17',
       'London SW1W', 'EC2A', 'TW7', 'SE21', 'SW11', 'E16', 'SW13', 'NW4',
       'W12', 'WC1A', 'N20', 'London SW8', 'WC1N', 'WC2E', 'SE8', 'NW2',
       'NW6', 'N21', 'SW5', 'W1D', 'Knightsbridge SW7',
       'East Finchley N2', 'TW9', 'Notting Hill W11', 'TW10', 'N1',
       'Belgravia SW1W', 'SW18', 'London NW8', 'Marylebone W1W',
       'Chelsea SW10', 'WC1H', 'London SW1X', 'N5', 'SW2', 'SW1V'],
      dtype=object)

In [130]:
zip_count = subset["ZIP"].dropna().value_counts().reset_index()
zip_count.columns = ["ZIP", "Count"]

top_30_zip_count = zip_count.head(30)

fig = px.bar(
    top_30_zip_count, 
    x="ZIP", 
    y="Count", 
    title="Top 30 ZIP Codes by Count",
    labels={"ZIP": "ZIP Code", "Count": "Count"},
    color="Count",
    color_continuous_scale="Viridis"
)

fig.update_layout(
    xaxis_title="ZIP Code",
    yaxis_title="Count",
    xaxis_tickangle=45,
    showlegend=False
)

fig.show()

In [131]:
postcode_coordinates = {
    # Central London
    'SW1W': (51.4998, -0.1429),  # Belgravia
    'SW1X': (51.5014, -0.1550),  # Mayfair
    'SW1A': (51.5007, -0.1246),  # Westminster
    'SW1E': (51.4943, -0.1431),  # Belgravia
    'SW1P': (51.4966, -0.1357),  # Victoria
    'SW1V': (51.4958, -0.1395),  # Pimlico
    
    # West London
    'W1B': (51.5167, -0.1378),  # Mayfair
    'W1D': (51.5141, -0.1320),  # Soho
    'W1G': (51.5193, -0.1463),  # Fitzrovia
    'W1H': (51.5221, -0.1409),  # Marylebone
    'W1K': (51.5114, -0.1465),  # Mayfair
    'W1S': (51.5087, -0.1436),  # Mayfair
    'W1U': (51.5166, -0.1504),  # Marylebone
    'W1W': (51.5225, -0.1376),  # Marylebone
    
    # North London
    'N1': (51.5230, -0.0984),
    'N2': (51.5906, -0.1715),  # East Finchley
    'N5': (51.5447, -0.1052),
    'N6': (51.5587, -0.1484),  # Highgate
    'N10': (51.5968, -0.1460),
    'N20': (51.6305, -0.2016),
    'N21': (51.6234, -0.1248),
    
    # Northwest London
    'NW1': (51.5229, -0.1607),
    'NW2': (51.5698, -0.2470),
    'NW3': (51.5490, -0.1770),  # Hampstead
    'NW4': (51.5978, -0.2440),
    'NW6': (51.5446, -0.2072),
    'NW7': (51.6238, -0.2547),
    'NW8': (51.5260, -0.1748),  # St John's Wood
    'NW11': (51.5830, -0.2098),
    
    # Southwest London
    'SW2': (51.4438, -0.1185),
    'SW3': (51.4850, -0.1750),  # Chelsea
    'SW5': (51.4886, -0.1950),
    'SW6': (51.4750, -0.2050),
    'SW7': (51.4950, -0.1750),  # South Kensington
    'SW8': (51.4830, -0.1230),  # Nine Elms
    'SW10': (51.4860, -0.1960),  # Chelsea
    'SW11': (51.4760, -0.1650),
    'SW13': (51.4730, -0.2570),
    'SW15': (51.4550, -0.2300),
    'SW16': (51.4230, -0.1190),
    'SW17': (51.4230, -0.1350),
    'SW18': (51.4530, -0.2170),
    'SW19': (51.4230, -0.2130),
    
    # East London
    'E14': (51.5049, -0.0210),
    'E16': (51.5080, 0.0300),
    'E20': (51.5085, -0.0130),
    
    # Southeast London
    'SE1': (51.5050, -0.0850),
    'SE8': (51.4910, -0.0360),
    'SE13': (51.4640, -0.0220),
    'SE21': (51.4430, -0.0840),
    
    # West Central London
    'WC1A': (51.5201, -0.1210),
    'WC1H': (51.5249, -0.1260),
    'WC1N': (51.5242, -0.1260),
    'WC2E': (51.5074, -0.1278),
    'WC2N': (51.5080, -0.1285),
    'WC2R': (51.5115, -0.1250),
    
    # Thames Valley/Outer London
    'TW1': (51.4450, -0.3220),
    'TW7': (51.4710, -0.3710),
    'TW9': (51.4570, -0.3050),
    'TW10': (51.4400, -0.3100),
    
    # Other notable locations
    'W2': (51.5170, -0.1700),
    'W8': (51.4950, -0.1950),  # Kensington
    'W9': (51.5260, -0.1850),
    'W11': (51.5090, -0.2050),  # Notting Hill
    'W12': (51.5060, -0.2250),
    'W14': (51.4950, -0.2050)
}


Observando la distribución descompensada por ZIP obtenemos una visualización geográfica para determinar el sesgo del dataset.

In [132]:
subset['Cleaned_ZIP'] = subset['ZIP'].str.extract(r'([A-Z]{1,2}\d{1,2}[A-Z]?)')[0]

postcode_counts = subset['Cleaned_ZIP'].value_counts().reset_index()
postcode_counts.columns = ['Postcode', 'Count']

postcode_counts['Latitude'] = postcode_counts['Postcode'].map(lambda x: postcode_coordinates.get(x, [None, None])[0])
postcode_counts['Longitude'] = postcode_counts['Postcode'].map(lambda x: postcode_coordinates.get(x, [None, None])[1])

postcode_counts = postcode_counts.dropna(subset=['Latitude', 'Longitude'])

fig = px.scatter_mapbox(
    postcode_counts, 
    lat='Latitude', 
    lon='Longitude', 
    hover_name='Postcode',
    color='Count',
    size='Count',
    color_continuous_scale='viridis',
    zoom=10, 
    height=600,
    mapbox_style="open-street-map"
)

fig.update_layout(
    title='London Postcodes - Frequency Heatmap',
    title_x=0.5,
    margin={"r":0,"t":50,"l":0,"b":0},
    mapbox=dict(
        center=dict(
            lat=51.5074,
            lon=-0.1278
        )
    )
)

fig.update_traces(
    hovertemplate='<b>%{hovertext}</b><br>Frequency: %{marker.color}<extra></extra>',
    selector=dict(mode='markers')
)

fig.show()

Existe una concentración de inmuebles del dataset en el centro-oeste de Londres, por lo que esta condición aplica a las hipotésis que vamos a poner a prueba.

#### **propertyType**

*   Descripción: Tipo de propiedad.
      *   Apartment
      *   Penthouse
      *   Flat
      *   House
      *   Terraced
      *   Detached

*   Tipo de variable: Categórica nominativa.

In [133]:
subset["propertyType"].describe()


count           735
unique            6
top       Apartment
freq            207
Name: propertyType, dtype: object

In [134]:
property_type_counts = subset['propertyType'].value_counts().reset_index()
property_type_counts.columns = ['Property Type', 'Count']

# Create a bar plot with the counts for each property type
fig = px.bar(
    property_type_counts, 
    x='Property Type',  # Use the cleaned column name
    y='Count', 
    title='Count of Properties by Type',
    labels={'Property Type': 'Property Type', 'Count': 'Number of Properties'},
    color='Property Type',  # Color bars by property type
    color_discrete_sequence=['#636EFA', '#EF553B', '#EF553B', '#636EFA', '#EF553B']  # Custom colors
)

# Customize layout and axes
fig.update_layout(
    xaxis_title='Property Type',
    yaxis_title='Number of Properties',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    bargap=0.1,  # Gap between bars
    template='plotly',  # Optional: for aesthetic improvements
)

# Show the plot
fig.show()


#### **propertyGlobalType**

*   Descripción: Tipo de propiedad principal.

      *   House
      *   Flat

*   Tipo de variable: Categórica nominativa.

In [135]:
subset["propertyGlobalType"].describe()


count      735
unique       2
top       Flat
freq       395
Name: propertyGlobalType, dtype: object

In [136]:
property_type_counts = subset['propertyGlobalType'].value_counts().reset_index()
property_type_counts.columns = ['Property Global Type', 'Count']

# Create a bar plot with the counts for each property type
fig = px.bar(
    property_type_counts, 
    x='Property Global Type',  # Use the cleaned column name
    y='Count', 
    title='Count of Properties by Type',
    labels={'Property Global Type': 'Property Global Type', 'Count': 'Number of Properties'},
    color='Property Global Type',  # Color bars by property type
    color_discrete_sequence=['#636EFA', '#EF553B', '#EF553B', '#636EFA', '#EF553B']  # Custom colors
)

# Customize layout and axes
fig.update_layout(
    xaxis_title='Property Global Type',
    yaxis_title='Number of Properties',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    bargap=0.1,  # Gap between bars
    template='plotly',  # Optional: for aesthetic improvements
)

# Show the plot
fig.show()


[MRDB]: La descriptiva y la presentación muy elaborada.

## 📈 **Analisis Bivariante**

### sizeSqFeetMax

In [166]:
fig = px.histogram(
    data_frame=subset,
    x='sizeSqFeetMax',
    color='propertyGlobalType',
    nbins=40,  # Number of bins
    title='Distribution of Property Sizes by Global Type',
    labels={'sizeSqFeetMax': 'Size (Sq. Feet)', 'propertyGlobalType': 'Property Global Type'},
    barmode='stack',  # Change to 'group' for side-by-side bars
    color_discrete_sequence=['#EF553B', '#636EFA', '#636EFA'],  # Blue for flats, Red for houses
    template='plotly',  # Optional: for aesthetic improvements
)

fig.update_layout(
    xaxis_title='Size (Sq. Feet)',
    yaxis_title='Frequency',
    bargap=0.1,  # Optional: gap between bars
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
)

fig.show()


### pricePerSqFoot(k)

In [111]:
fig = px.histogram(
    data_frame=subset,
    x='pricePerSqFoot(k)',
    color='propertyGlobalType',
    nbins=20,  # Number of bins
    title='Distribution of Price Per Sq. Foot by Global Type',
    labels={'pricePerSqFoot(k)': 'Price Per Sq. Foot', 'propertyGlobalType': 'Property Global Type'},
    barmode='stack',  # Stack bars
    color_discrete_sequence=['#EF553B', '#636EFA', '#636EFA'],  # Blue for flats, Red for houses
    template='plotly',  # Optional: for aesthetic improvements,
)

fig.update_layout(
    xaxis_title='Price Per Sq. Foot (in thousands)',
    yaxis_title='Frequency',
    bargap=0.1,  # Optional: gap between bars
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
)

fig.show()

### price (M)

#### Distribution per propertyGlobalType

In [109]:
fig = px.histogram(
    data_frame=subset,
    x='price(M)',
    color='propertyGlobalType',
    nbins=50,  # Number of bins
    title='Distribution of Property Prices by Global Type',
    labels={'price(M)': 'Price (in millions)', 'propertyGlobalType': 'Property Global Type'},
    barmode='stack',  # Change to 'group' for side-by-side bars
    template='plotly',  # Optional: for aesthetic improvements
    color_discrete_sequence=['#EF553B', '#636EFA', '#636EFA']  # Blue for flats, Red for houses
)

fig.update_layout(
    xaxis_title='Price (in millions)',
    yaxis_title='Frequency',
    bargap=0.1,  # Optional: gap between bars
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    legend_title='Property Global Type',
)

fig.show()


#### pricePerSqFoot (k)

##### propertyType

In [108]:
fig = px.scatter(subset, y="price(M)", x="pricePerSqFoot(k)", color="propertyType")
fig.show()

##### propertyGlobalType

In [107]:
fig = px.scatter(subset, y="price(M)", x="pricePerSqFoot(k)", color="propertyGlobalType")
fig.show()

#### sizeSqFeetMax

##### propertyType

In [137]:
fig = px.scatter(subset, y="price(M)", x="sizeSqFeetMax", color="propertyType")
fig.show()

##### propertyGlobalType

In [138]:
fig = px.scatter(subset, y="price(M)", x="sizeSqFeetMax", color="propertyGlobalType")
fig.show()

#### pricePerSqFoot(k)

##### propertyType

In [101]:
fig = px.scatter(subset, y="price(M)", x="pricePerSqFoot(k)", color="propertyType")
fig.show()

##### propertyGlobalType

In [139]:
fig = px.scatter(subset, y="price(M)", x="pricePerSqFoot(k)", color="propertyGlobalType")
fig.show()

#### bedrooms

##### propertyType

In [104]:
fig = px.violin(
    subset, 
    x='bedrooms',
    y='price(M)',
    color='propertyType',
    title='Price Distribution by Number of Bedrooms and Property Type',
    labels={'bedrooms': 'Number of Bedrooms', 'price(M)': 'Price (in millions)'},
    box=False,
    points='all',

)

fig.update_layout(
    xaxis_title='Number of Bedrooms',
    yaxis_title='Price (in millions)',
    template='plotly',
    xaxis_range=[0, 13],
    yaxis_range=[0, 80],
    xaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1
    ),
)

fig.show()


##### propertyGlobalType

In [103]:
fig = px.violin(
    subset, 
    x='bedrooms',
    y='price(M)',
    color='propertyGlobalType',
    title='Price Distribution by Number of Bedrooms and Property Global Type',
    labels={'bedrooms': 'Number of Bedrooms', 'price(M)': 'Price (in millions)'},
    box=False,
    points='all',

)

fig.update_layout(
    xaxis_title='Number of Bedrooms',
    yaxis_title='Price (in millions)',
    template='plotly',
    xaxis_range=[0, 13],
    yaxis_range=[0, 80],
    xaxis=dict(
        tickmode='linear',
        tick0=1,
        dtick=1
    ),
)

fig.show()


#### bathrooms

##### propertyType

In [105]:
fig = px.violin(
    subset, 
    x='bathrooms',
    y='price(M)',
    color='propertyType',
    title='Price Distribution by Number of Bathrooms and Property Type',
    labels={'bathrooms': 'Number of Bathrooms', 'price(M)': 'Price (in millions)'},
    box=False,
    points='all'
)

fig.update_layout(
    xaxis_title='Number of Bedrooms',
    yaxis_title='Price (in millions)',
    template='plotly',
    xaxis_range=[0, 13],  # Limit x-axis range (if needed)
    yaxis_range=[0, 80], # Limit y-axis range (if needed)
    xaxis=dict(
        tickmode='linear',   # Use linear ticks
        tick0=1,             # Start ticks at 1
        dtick=1              # Step size of 1
    ),
)

fig.show()


##### propertyGlobalType

In [106]:
fig = px.violin(
    subset, 
    x='bathrooms',
    y='price(M)',
    color='propertyGlobalType',
    title='Price Distribution by Number of Bathrooms and Property Global Type',
    labels={'bathrooms': 'Number of Bathrooms', 'price(M)': 'Price (in millions)'},
    box=False,
    points='all'
)

fig.update_layout(
    xaxis_title='Number of Bedrooms',
    yaxis_title='Price (in millions)',
    template='plotly',
    xaxis_range=[0, 13],  # Limit x-axis range (if needed)
    yaxis_range=[0, 80], # Limit y-axis range (if needed)
    xaxis=dict(
        tickmode='linear',   # Use linear ticks
        tick0=1,             # Start ticks at 1
        dtick=1              # Step size of 1
    ),
)

fig.show()


[MRDB]: Aquí os faltaría un poco de comentarios

## 🏹 **Estimación puntual y por intervalos.**

* Estimación puntual:
    * Media poblacional ~ £11M
* Por intervalos:
    * Media poblacional = £10.9 ~ £11.8 al 95%

Para casas de:

* Centro-oeste de Londres.
* Más de 6,5M.


### Media poblacional

#### Estimación puntual

Estimacion puntual de la media poblacional.

* Del precio de los inmuebles.
* Mediante el uso de la media muestral.

In [140]:
subset["price(M)"].mean()

np.float64(10.958573401360544)

**Podemos asumir que la media poblacional ronda los 11M de libras, al igual que la media muestral.**

Para casas de:

* Centro-oeste de Londres.
* Más de 6,5M.

#### Estimación por intervalos

Estimación de la media poblacional por intervalos de confianza del 95%.

In [142]:
st.t.interval(confidence=0.95, df=len(data["price(M)"])-1, loc=np.mean(data["price(M)"]), scale=st.sem(data["price(M)"]))

(np.float64(10.87792233937227), np.float64(11.788304760745516))

**Se puede afirmar al 95% que la media poblacional del precio de las casas está entre 10.9 y 11.8 millones de libras.**

Para casas de:

* Centro-oeste de Londres.
* Más de 6,5M.

## 🔮 **Hipótesis**

Rechazamos la hipótesis nula:
𝜇 casas > 𝜇 pisos al 95%

Rechazamos la hipótesis nula:
𝜇 casas > 11M

### House > Flat

* H₀: La media de los precios de las casas es mayor que la de los pisos
    * ( 𝜇 casas > 𝜇 pisos )

* H₁: La media de los precios de las casas es menor o igual que la de los pisos
    * ( 𝜇 casas ≤ 𝜇 pisos )

In [143]:
house_df =  subset[subset['propertyGlobalType'] == "House"]
flat_df =  subset[subset['propertyGlobalType'] == "Flat"]

In [ ]:
t_stat, p_value = stats.ttest_ind(house_df["price(M)"], flat_df["price(M)"], alternative='greater', equal_var=False)

print(f"Estadístico t: {t_stat}")
print(f"Valor p: {p_value}")

alpha = 0.05
if p_value < alpha:
    print("\nRechazamos la hipótesis nula:\n𝜇 casas > 𝜇 pisos al 95%")
else:
    print("\nNo podemos rechazar la hipótesis nula.")

Estadístico t: 6.059136130387427
Valor p: 1.3228725788820076e-09

Rechazamos la hipótesis nula:
𝜇 casas > 𝜇 pisos al 95%


### House > 11M

* H₀: La media de los precios de las casas es mayor a 11M
    * ( 𝜇 casas > 11M )

* H₁: La media de los precios de las casas es menor o igual a 11M
    * ( 𝜇 casas ≤ 11M )

In [163]:
mi = subset["price(M)"].mean()

t_stat, p_value = stats.ttest_1samp(house_df["price(M)"], mi)

p_value_unilateral = p_value / 2 if t_stat > 0 else 1 - (p_value / 2)

print(f"Estadístico t: {t_stat}")
print(f"Valor p unilateral: {p_value_unilateral}")

alpha = 0.05
if p_value < alpha:
    print("\nRechazamos la hipótesis nula:\n𝜇 casas > 11M")
else:
    print("\nNo podemos rechazar la hipótesis nula.")

Estadístico t: 3.670060570309344
Valor p unilateral: 0.00014078069800749874

Rechazamos la hipótesis nula:
𝜇 casas > 11M


[MRDB]: Aquí un poc de comentario. El pvalor por debajo del nivel de significación paraece indicar con evidencia estadística suficiente que la media del precio de casa es menor o igual a 11M. Tenéis que pensar que esto no lo lee un estadistico y se le dice a alguien "Rechazamos la hipótesis nula: 𝜇 casas > 11M" quizás no está entendiendo nada

## 📈 **Anova**

### Apartment = Penthouse = Flat

* H₀: La media de los precios de los distintos tipos de piso es igual
    * ( 𝜇 apartments == 𝜇 penthouse == 𝜇 flat )

* H₁: Al menos una de las medias de los precios de los distintos tipos de piso es diferente del resto.

In [165]:
Apartmente_df =  subset[subset['propertyType'] == "Apartment"]
Penthouse_df =  subset[subset['propertyType'] == "Penthouse"]
Flat_df =  subset[subset['propertyType'] == "Flat"]

f_stat, p_value = stats.f_oneway(Apartmente_df["price(M)"], Penthouse_df["price(M)"], Flat_df["price(M)"]) #f_oneway realiza un ANOVA de una vía.

alpha = 0.05
if p_value < alpha:
    print("\nRechazamos la hipótesis nula.\nHay diferencias significativas entre las medias.")
else:
    print("\nNo podemos rechazar la hipótesis nula.\nLas medias son estadísticamente iguales.")


Rechazamos la hipótesis nula.
Hay diferencias significativas entre las medias.


[MRDB]: La primera arte del trabajod es de sobresaliente, la segunda es de "suficiente". Os faltan comentarios, más análisis de la segunda parte. Parte del trabajo es la organización de las tareas y habéis invertido mucho tiempo en unas y poco tiempoo en otras. Es preferibe, hacer un informe que esté completo auqneu menso trabajado en unas de las partes. Os faltan conclusiones y trasaldo de los análisis a un lenguaje de informe, guienado al lector en los resultados. Nota un 6,5. Aunque en relaidad no habéis conseguido el objetivo del informe, simplemente habéis descrito muy bien la bas de datos que no era lo único que había que hacer.

Os deseo unas felices fiestas y un feliz año nuevo. Suerte con lo que os viene, id con calma y mejorando poco a poco, a disfrutar del camino del aprendizaje y os deseo lo mejor!